# 1. Import libraries

In [ ]:
import tensorflow as tf
from konlpy.tag import Twitter
from collections import Counter
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt

keras = tf.keras
t = Okt()

# 2. fit tokenizer to our datasets

In [ ]:
from vectorizer import BaseVectorizer
tokenizer = BaseVectorizer(t.morphs)

In [ ]:
# 데이터 셋 불러온다
df = pd.read_csv('train_intent.csv',encoding='CP949')

In [ ]:
# 잘 불러와졌나 확인
df[0:1]

In [ ]:
# csv 파일의 1행1열의 값을 넣으면 된다
# 이 코드에서 불러온 train_intent는 1행1열이 question임
tokenizer.fit(df['question'].values)

In [ ]:
# 문장을 단어별로 나누고 단어에 id 부여
tokenizer.vocabulary_

# 3. data preprocessing

In [ ]:
# intent의 레이블과 id를 변환해주는 딕셔너리
label_to_id = {t:i for i,t in enumerate(df.intent.unique())}
id_to_label = {i:t for i,t in enumerate(df.intent.unique())}

In [ ]:
print(label_to_id)

In [ ]:
# 단어 단위로 나눈다
# 만약 "인터넷 좀 켜줘" 였다면 인터넷, 좀, 켜, 줘 정도로 나누고 
# 최대길이 20 이하의 부분은 _PAD_ 로처리한다
MAX_LENGTH = 20

def tokenize_and_filter(sentences, labels):
    inputs, outputs = [], []
  
    for sentence, label in zip(sentences, labels):
        # tokenize sentence
        tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
        # check tokenized sentence max length
        if len(tokenized_sentence) <= MAX_LENGTH:
            inputs.append(tokenized_sentence)
            outputs.append(label_to_id[label])
  
      # pad tokenized sentences
        padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
            inputs, maxlen=MAX_LENGTH, padding='post', 
            value = tokenizer.vocabulary_['_PAD_']) # value = 0
  
    return padded_inputs, outputs

In [ ]:
inputs, outputs = tokenize_and_filter(df.question, df.intent)

In [ ]:
print('encoded input : ', inputs[0], 'label : ', outputs[0], 'original input sentence : ', tokenizer.decode_from_list(inputs[0]))

In [ ]:
# 데이터셋을 섞고 배치사이즈만큼 나눈다
BATCH_SIZE = 16
BUFFER_SIZE = 7836

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for x, y in dataset.take(1):
    print(x, y)
    print('-----------------------------------------------')
    print(x.shape, y.shape)

# 4. model design

In [ ]:
print(len(label_to_id.values()))

In [ ]:
# 모델을 디자인한다 
# 간단하게 임베딩 후 LSTM 으로 처리하고 활성화함수로는 relu를 사용했다
def get_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.n_vocabs, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(label_to_id.values()), activation='softmax')
])
    LEARNING_RATE = 0.0001
    model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=[tf.keras.metrics.sparse_categorical_accuracy])
    return model

In [ ]:
model = get_model()

In [ ]:
# 학습시킴
model.fit(dataset, epochs=20)

In [ ]:
# 예측 할 문장들을 단어 단위로 쪼개서 vocabulary에 있는 인덱스로 key-value 설정한다

def question_processing(sentences):
    inputs = []
    for sentence in sentences:
        tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
        if len(tokenized_sentence) <= MAX_LENGTH:
            inputs.append(tokenized_sentence)
        else:
            print('입력이 너무 길어요.')

    padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    inputs, maxlen=MAX_LENGTH, padding='post', 
    value = tokenizer.vocabulary_['_PAD_']) # value = 0
    return padded_inputs

In [ ]:
input_sentence = question_processing(['인터넷 한 번 켜봐', 
                                      '그림 그리게 인터넷 켜줘',
                                      '인터넷이나 한 번 해볼까',
                                      '음.. 그림 그려볼까',
                                      '인터넷 ㄱㄱ'
                                     ])

In [ ]:
# 예측 해본다
model.predict(input_sentence)

In [ ]:
# 예측값을 id값으로 출력
prediction = np.argmax(model.predict(input_sentence), axis=1)
print(prediction)

In [ ]:
# id 값을 레이블 값으로 
for p in prediction:
    print(id_to_label[p])

In [ ]:
# # 모델 저장하기
# model.save('intent_model.h5')
# 모델 불러오기
tempmodel = keras.models.load_model('intent_model.h5')
tempmodel.summary()

In [ ]:
tempmodel.predict(input_sentence)

In [ ]:
prediction = list(np.argmax(tempmodel.predict(input_sentence), axis=1))

In [ ]:
prediction